# Workshop Tutorial: Constraints

In this tutorial we will learn about constraints between Parameters and the default parameterization of a binary system.

This interactive workshop tutorial covers many of the same topics as the corresponding online tutorials:

* [Constraints](http://phoebe-project.org/docs/latest/tutorials/constraints.ipynb)
* [Advanced: Constraints and Changing Hierarchies](http://phoebe-project.org/docs/latest/tutorials/constraints_hierarchies.ipynb)
* [Advanced: Built-In Constraints](http://phoebe-project.org/docs/latest/tutorials/constraints_builtin.ipynb)

# Setup

In [1]:
import phoebe
from phoebe import u,c

In [2]:
logger = phoebe.logger(clevel='WARNING')

In [3]:
b = phoebe.default_binary()

# Constraints

As we saw in the last exercise, there are 5 Parameters with a qualifier of `incl`.

In [4]:
print(b.filter(qualifier='incl'))

ParameterSet: 5 parameters
C          incl@primary@component: 90.0 deg
C        incl@secondary@component: 90.0 deg
            incl@binary@component: 90.0 deg
          incl@primary@constraint: {incl@binary@component} + {pitch@primary@component}
        incl@secondary@constraint: {incl@binary@component} + {pitch@secondary@component}


Note that here the previously-mentioned twig-syntax is shown to show as much information as possible about the Parameters.

Three of these are because there are inclinations defined for the orbit as well as each of the two stars ('primary' and 'secondary').  These three Parameters all have `context='component'`.

In [5]:
print(b.filter(qualifier='incl', context='component'))

ParameterSet: 3 parameters
C          incl@primary@component: 90.0 deg
C        incl@secondary@component: 90.0 deg
            incl@binary@component: 90.0 deg


The other inclinations of the stars are (by default) *constrained* to be the same as the inclination of the orbit (i.e., an aligned system).  We can see this by the `C` in the output (to the left of the twigs) above.

In [6]:
b.set_value(qualifier='incl', context='component', component='primary', value=80)

ValueError: cannot change the value of a constrained parameter.  This parameter is constrained by 'incl@binary@component, pitch@primary@component'

Because the inclination of the primary is constrained, an error will be raised if we try to change the value directly.  The error message tells us that the inclination of the primary is constrained by the inclination of the binary and the pitch of the primary.  We can also get this information by accessing the `constrained_by` attribute of the Parameter

In [7]:
b.get_parameter(qualifier='incl', context='component', component='primary').constrained_by

[<Parameter: incl=90.0 deg | keys: description, value, quantity, default_unit, limits, visible_if, copy_for, readonly, advanced, latexfmt>,
 <Parameter: pitch=0.0 deg | keys: description, value, quantity, default_unit, limits, visible_if, copy_for, readonly, advanced, latexfmt>]

The other two Parameters with `qualifier='incl'` are the constraints themselves and have `context='constraint'`.  Filtering on this will give us the equation used to calculate the value of the constrained parameter.

In [8]:
print(b.filter(qualifier='incl', context='constraint'))

ParameterSet: 2 parameters
          incl@primary@constraint: {incl@binary@component} + {pitch@primary@component}
        incl@secondary@constraint: {incl@binary@component} + {pitch@secondary@component}


In [9]:
b.get_parameter(qualifier='incl', context='constraint', component='primary')

<ConstraintParameter: {incl@primary@component} = {incl@binary@component} + {pitch@primary@component} (solar units) => 90.0 deg>

Here we see that this is a simple constraint: the inclination of the primary star is being *constrained* to be exactly that of the inclination of the binary orbit (since the `pitch` is set to zero).  If we change the inclination of the orbit, the inclinations of the 'primary' and 'secondary' stars will immediately update to reflect that change.

In [10]:
b.set_value(qualifier='incl', component='binary', value=80)

In [11]:
print(b.filter(qualifier='incl', context='component'))

ParameterSet: 3 parameters
C          incl@primary@component: 80.0 deg
C        incl@secondary@component: 80.0 deg
            incl@binary@component: 80.0 deg


And similarly, if we change the pitch of the primary, then the inclination of the primary will immediately update to reflect the change.

In [12]:
b.set_value(qualifier='pitch', component='primary', value=-5)

In [13]:
print(b.filter(qualifier='incl', context='component'))

ParameterSet: 3 parameters
C          incl@primary@component: 75.0 deg
C        incl@secondary@component: 80.0 deg
            incl@binary@component: 80.0 deg


Other constraints are a little more complicated.

In [14]:
b.get_parameter(qualifier='asini', component='binary', context='constraint')

<ConstraintParameter: {asini@binary@component} = {sma@binary@component} * (sin({incl@binary@component})) (solar units) => 5.2194810909647025 solRad>

In [15]:
print("asini: {}, sma: {}, incl: {}".format(
    b.get_value(qualifier='asini', component='binary', context='component'),
    b.get_value(qualifier='sma', component='binary', context='component'),
    b.get_value(qualifier='incl', component='binary', context='component')))

asini: 5.2194810909647025, sma: 5.3, incl: 80.0


In [16]:
b.set_value(qualifier='sma', component='binary', context='component', value=10.0)

In [17]:
print("asini: {}, sma: {}, incl: {}".format(
    b.get_value(qualifier='asini', component='binary', context='component'),
    b.get_value(qualifier='sma', component='binary', context='component'),
    b.get_value(qualifier='incl', component='binary', context='component')))

asini: 9.84807753012208, sma: 10.0, incl: 80.0


# Re-Parameterizing or "Flipping" Constraints

In the default binary, there are a significant number of constrained Parameters.

In [18]:
print(b.filter(context='constraint').qualifiers)

['ebv', 'freq', 'logg', 'irrad_frac_lost_bol', 'asini', 't0_perpass', 't0_ref', 'mean_anom', 'ecosw', 'esinw', 'teffratio', 'requivratio', 'requivsumfrac', 'mass', 'sma', 'period', 'incl', 'long_an', 'requiv_max']


Let's look at mass, which is _constrained_ by default according to Kepler's third law.

In [19]:
print(b.get_parameter('mass', component='primary', context='component'))

Parameter: mass@primary@component
                       Qualifier: mass
                     Description: Mass
                           Value: 6.708982151748291 solMass
                  Constrained by: sma@binary@component, period@binary@component, q@binary@component
                      Constrains: logg@primary@component, mass@secondary@component
                      Related to: requiv@primary@component, logg@primary@component, sma@binary@component, period@binary@component, q@binary@component, mass@secondary@component



In [20]:
print(b.get_parameter('mass', component='primary', context='constraint'))

Constrains (qualifier): mass
Expression in solar units (value): (39.478418 * ({sma@binary@component} ** 3.000000)) / ((({period@binary@component} ** 2.000000) * ({q@binary@component} + 1.000000)) * 2942.206217504419328179210424423218)
Current Result (result): 6.708982151748291 solMass


Here we see the 4 parameters that are involved in Kepler's third law. PHOEBE allows you to freely set 3 of these 4 (sma, period, q) and automatically uses these values to compute that mass.

However, let's say that you wanted to set the mass (perhaps you know the mass, but don't know the semi-major axis as well). This can be done via the `flip_constraint` method. The easiest way to use this correctly is to make sure our keywords return the correct Constraint Parameter via `get_constraint` and then use `flip_constraint`.

In [21]:
b.get_constraint(qualifier='mass', component='primary')

<ConstraintParameter: {mass@primary@component} = (39.478418 * ({sma@binary@component} ** 3.000000)) / ((({period@binary@component} ** 2.000000) * ({q@binary@component} + 1.000000)) * 2942.206217504419328179210424423218) (solar units) => 6.708982151748291 solMass>

Now we can use `flip_constraint` just add `solve_for='sma'` to "flip" this constraint to solve for 'sma' instead of 'mass'.

In [22]:
b.flip_constraint(qualifier='mass', component='primary', solve_for='sma')

<ConstraintParameter: {sma@binary@component} = (((({mass@primary@component} * ({period@binary@component} ** 2.000000)) * ({q@binary@component} + 1.000000)) * 2942.206217504419328179210424423218) / 39.478418) ** (1./3) (solar units) => 9.999999999999998 solRad>

Now we're allowed to set the mass and we'll see that the value of sma is automatically computed.

In [23]:
b.set_value('mass', component='primary', value=1.2)

In [24]:
b.get_value('sma', component='binary', context='component')

5.634320356809353

# Exercise

How is q defined: is it Mprimary/Msecondary or Msecondary/Mprimary?

Hint: there are (at least) 2 ways to do this.  Try first by looking through the equations of the constraints if you feel a bit daring.  You can also change the value of q and see how the resulting constrained masses react.

Flipping constraints could be particularly useful if you have an observational constraint on 'asini' (say from the amplitude of RVs) and want to leave asini fixed as you fit for the inclination.  Flip the constraint so it is possible to adjust the values of both 'asini' and 'incl'.  (**NOTE** you may want to either start from a fresh bundle or re-flip the Kepler's third law constraint back to solve for mass first).

Now that you can change the value of 'asini', set it to 20 (solar radii... we'll talk about units in the next tutorial), adjust the inclination, and show that 'sma' is adjusting automatically to conserve 'asini'.